In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, "/home/ylu/project")
from utils import *

sys.path.insert(0, "../../scripts/methods/")
from my_moscot import moscot_align, moscot_align_ref
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

_XSERVTransSocketUNIXCreateListener: ...SocketCreateListener() failed
_XSERVTransMakeAllCOTSServerListeners: server already running
(EE) 
Fatal server error:
(EE) Cannot establish any listening sockets - Make sure an X server isn't already running(EE) 


In [2]:
data_folder = "./results/split_slices_CCF/"
results_folder = "./results/Moscot/"
figures_folder = "./results/figures/Moscot"
Path(results_folder).mkdir(parents=True, exist_ok=True)
Path(figures_folder).mkdir(parents=True, exist_ok=True)

In [3]:
import re
def list_and_sort_files_by_number(directory_path, suffix):
    def extract_number(filename):
        match = re.search(r'\d+', filename)
        return int(match.group()) if match else float('inf')
    try:
        files = [f for f in os.listdir(directory_path) if f.endswith(suffix)]
        sorted_files = sorted(files, key=extract_number)
        return sorted_files
    except FileNotFoundError:
        return f"The directory {directory_path} does not exist."
    except NotADirectoryError:
        return f"The path {directory_path} is not a directory."
    except PermissionError:
        return f"Permission denied to access {directory_path}."

directory_path = data_folder
sorted_files = list_and_sort_files_by_number(directory_path, '.h5ad')

In [4]:
## load the data
from tqdm import tqdm
slices = []
for f in tqdm(sorted_files):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f)))

100%|███████████████████████████| 129/129 [00:22<00:00,  5.81it/s]


In [5]:
spatial_key = "spatial_2D"
anno_key = "cell_type"

In [6]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [7]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [8]:
## Perform the Moscot alignment
sampling_num = 20000
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    ratio = sampling_num * 2 / (slice1.n_obs + slice2.n_obs)
    sampline_idx1 = np.random.choice(slice1.shape[0], int(slice1.shape[0] * ratio), replace=False) if slice1.shape[0] > int(slice1.shape[0] * ratio) else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], int(slice2.shape[0] * ratio), replace=False) if slice2.shape[0] > int(slice2.shape[0] * ratio) else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis = moscot_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        device="cuda",
        verbose=False,
        alpha=0.5,
    )
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                     | 0/128 [00:00<?, ?it/s]

INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  


/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseBrain_Zhuang/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Normalizing spatial coordinates of `y`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(4731, 5268)].                                          


2024-07-15 19:27:24.954933: W external/xla/xla/service/gpu/nvptx_compiler.cc:760] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
  1%|▏                            | 1/128 [00:12<26:40, 12.60s/it]

...

 99%|██████████████████████████▊| 127/128 [23:03<00:11, 11.68s/it]

INFO     Computing pca with `n_comps=30` for `xy` using `adata.X`                                                  


/home/ylu/project/Benchmark_alignment_methods/notebooks/AllenMouseBrain_Zhuang/../../scripts/methods/my_moscot.py:56: ImplicitModificationWarning: Setting element `.obsm['aligned_spatial']` of view, initializing view as actual.
  m.obsm[key_added] = m.obsm[spatial_key]


INFO     Normalizing spatial coordinates of `x`.                                                                   
INFO     Normalizing spatial coordinates of `y`.                                                                   
INFO     Solving `1` problems                                                                                      
INFO     Solving problem OTProblem[stage='prepared', shape=(5525, 4474)].                                          


100%|███████████████████████████| 128/128 [23:14<00:00, 10.90s/it]
